# Knowledge Structure Graph Generator

## 1. DB연결

In [1]:
import pymysql

In [2]:
juso_db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='127.0.0.1', 
    db='test', 
    charset='utf8'
)

In [3]:
cursor = juso_db.cursor(pymysql.cursors.DictCursor)

In [4]:
sql = "SELECT * FROM knowledge_structure;"
cursor.execute(sql)
result = cursor.fetchall()

In [10]:
import pandas as pd

result = pd.DataFrame(result)
print("------------------------------지식구조 목록 리스트------------------------------")
print(result) #지식구조 목록 리스트화

------------------------------지식구조 목록 리스트------------------------------
  DOC_ID COT_MODE                                KNOWLEDGE_STRUCTURE
0   Hemp      scs  hemp,paper,5.7\nus,varieti,5.76\nhemp,us,5.78\...
1     햄프      scs  필요,확장,1.0\n외부,링크,1.0\n식별자,국가,1.0\n식별자,과거,1.0\n...


In [8]:
knowledge_structure = result['KNOWLEDGE_STRUCTURE']

## 2. 데이터 선택

In [13]:
data = knowledge_structure[1] #인덱스만 변경하기
print(data)

필요,확장,1.0
외부,링크,1.0
식별자,국가,1.0
식별자,과거,1.0
국가,과거,1.0
사용,램프,1.43
내용,식별자,2.1
내용,국가,2.1
내용,과거,2.1
분류,속씨식물군,2.19
문서,필요,2.3
문서,확장,2.3
외부,램프,2.76
파킨슨병,착색제,2.76
세계,착색제,2.76
세계,한때,2.76
링크,램프,2.76
편집,외부,3.54
편집,링크,3.54
삼과,속씨식물군,3.54
분류,식별자,3.79
분류,국가,3.79
분류,과거,3.79
외부,관련,4.0
관련,링크,4.0
문서,내용,4.55
편집,내용,5.0
사용,착색제,5.89



In [18]:
import re

contents = re.split('\n', data)
contents = contents[:-1] #맨 끝 데이터는 공백

In [22]:
#노드 추가
for content in contents:
    value = re.split(',', content) # ,단위로 구분하여 리스트화
    print(value)

['필요', '확장', '1.0']
['외부', '링크', '1.0']
['식별자', '국가', '1.0']
['식별자', '과거', '1.0']
['국가', '과거', '1.0']
['사용', '램프', '1.43']
['내용', '식별자', '2.1']
['내용', '국가', '2.1']
['내용', '과거', '2.1']
['분류', '속씨식물군', '2.19']
['문서', '필요', '2.3']
['문서', '확장', '2.3']
['외부', '램프', '2.76']
['파킨슨병', '착색제', '2.76']
['세계', '착색제', '2.76']
['세계', '한때', '2.76']
['링크', '램프', '2.76']
['편집', '외부', '3.54']
['편집', '링크', '3.54']
['삼과', '속씨식물군', '3.54']
['분류', '식별자', '3.79']
['분류', '국가', '3.79']
['분류', '과거', '3.79']
['외부', '관련', '4.0']
['관련', '링크', '4.0']
['문서', '내용', '4.55']
['편집', '내용', '5.0']
['사용', '착색제', '5.89']


## 3. Data JavaScript File 생성

### 파일 2개 생성
* keyword.js : data(nodes, edges)
* keyword.html : graph

### 01. keyword.js

In [23]:
import networkx as nx

In [24]:
G = nx.Graph()

In [25]:
# 에지리스트를 그래프에 넣기 : 그래프에 입력된 노드 확인용
for content in contents:
    tmp = re.split(',', content)
    G.add_edge(tmp[0], tmp[1], weight = tmp[2])

In [26]:
# 파일 저장용 nodes, edges
graph_nodes = "var nodes = "
graph_edges = "var edges = "

# 리스트 생성용
num = 0
nodes_list = []
reverse_graph = dict()

for name in G.nodes():
    # 딕셔너리 1개씩 생성하여 넣기
    tmp = dict()
    tmp['id'] = num
    tmp['label'] = name
    nodes_list.append(tmp)   # {id: 0, label: 'hemp'}형태
    
    reverse_graph[name] = num   # {'hemp':0, 'cannabi':1}형태, 목적: 'hemp'검색시 바로 노드 번호 확인(편의성 목적)
    num += 1

In [27]:
graph_nodes += str(nodes_list) + ";"
print(graph_nodes)

var nodes = [{'id': 0, 'label': '필요'}, {'id': 1, 'label': '확장'}, {'id': 2, 'label': '외부'}, {'id': 3, 'label': '링크'}, {'id': 4, 'label': '식별자'}, {'id': 5, 'label': '국가'}, {'id': 6, 'label': '과거'}, {'id': 7, 'label': '사용'}, {'id': 8, 'label': '램프'}, {'id': 9, 'label': '내용'}, {'id': 10, 'label': '분류'}, {'id': 11, 'label': '속씨식물군'}, {'id': 12, 'label': '문서'}, {'id': 13, 'label': '파킨슨병'}, {'id': 14, 'label': '착색제'}, {'id': 15, 'label': '세계'}, {'id': 16, 'label': '한때'}, {'id': 17, 'label': '편집'}, {'id': 18, 'label': '삼과'}, {'id': 19, 'label': '관련'}];


In [28]:
print(reverse_graph) # graph_nodes를 뒤집어서 저장, 예를 들어  label명 : id명

{'필요': 0, '확장': 1, '외부': 2, '링크': 3, '식별자': 4, '국가': 5, '과거': 6, '사용': 7, '램프': 8, '내용': 9, '분류': 10, '속씨식물군': 11, '문서': 12, '파킨슨병': 13, '착색제': 14, '세계': 15, '한때': 16, '편집': 17, '삼과': 18, '관련': 19}


In [29]:
edges_list = []

# edges는 {'hemp', 'cannabi', '3.21'}형태
for edge in G.edges(data='weight'):
    tmp = dict()
    tmp['from']  = reverse_graph.get(edge[0])  # 'hemp'검색시 인덱스값
    tmp['to']    = reverse_graph.get(edge[1])  # 'cannabi'검색시 인덱스값
    tmp['label'] = edge[2]                    # 가중치 저장

    edges_list.append(tmp)

In [30]:
graph_edges += str(edges_list) + ";"
print(graph_edges)

var edges = [{'from': 0, 'to': 1, 'label': '1.0'}, {'from': 0, 'to': 12, 'label': '2.3'}, {'from': 1, 'to': 12, 'label': '2.3'}, {'from': 2, 'to': 3, 'label': '1.0'}, {'from': 2, 'to': 8, 'label': '2.76'}, {'from': 2, 'to': 17, 'label': '3.54'}, {'from': 2, 'to': 19, 'label': '4.0'}, {'from': 3, 'to': 8, 'label': '2.76'}, {'from': 3, 'to': 17, 'label': '3.54'}, {'from': 3, 'to': 19, 'label': '4.0'}, {'from': 4, 'to': 5, 'label': '1.0'}, {'from': 4, 'to': 6, 'label': '1.0'}, {'from': 4, 'to': 9, 'label': '2.1'}, {'from': 4, 'to': 10, 'label': '3.79'}, {'from': 5, 'to': 6, 'label': '1.0'}, {'from': 5, 'to': 9, 'label': '2.1'}, {'from': 5, 'to': 10, 'label': '3.79'}, {'from': 6, 'to': 9, 'label': '2.1'}, {'from': 6, 'to': 10, 'label': '3.79'}, {'from': 7, 'to': 8, 'label': '1.43'}, {'from': 7, 'to': 14, 'label': '5.89'}, {'from': 9, 'to': 12, 'label': '4.55'}, {'from': 9, 'to': 17, 'label': '5.0'}, {'from': 10, 'to': 11, 'label': '2.19'}, {'from': 11, 'to': 18, 'label': '3.54'}, {'from': 

In [31]:
graph_data = graph_nodes + "\n\n" + graph_edges

In [34]:
print(graph_data)

var nodes = [{'id': 0, 'label': '필요'}, {'id': 1, 'label': '확장'}, {'id': 2, 'label': '외부'}, {'id': 3, 'label': '링크'}, {'id': 4, 'label': '식별자'}, {'id': 5, 'label': '국가'}, {'id': 6, 'label': '과거'}, {'id': 7, 'label': '사용'}, {'id': 8, 'label': '램프'}, {'id': 9, 'label': '내용'}, {'id': 10, 'label': '분류'}, {'id': 11, 'label': '속씨식물군'}, {'id': 12, 'label': '문서'}, {'id': 13, 'label': '파킨슨병'}, {'id': 14, 'label': '착색제'}, {'id': 15, 'label': '세계'}, {'id': 16, 'label': '한때'}, {'id': 17, 'label': '편집'}, {'id': 18, 'label': '삼과'}, {'id': 19, 'label': '관련'}];

var edges = [{'from': 0, 'to': 1, 'label': '1.0'}, {'from': 0, 'to': 12, 'label': '2.3'}, {'from': 1, 'to': 12, 'label': '2.3'}, {'from': 2, 'to': 3, 'label': '1.0'}, {'from': 2, 'to': 8, 'label': '2.76'}, {'from': 2, 'to': 17, 'label': '3.54'}, {'from': 2, 'to': 19, 'label': '4.0'}, {'from': 3, 'to': 8, 'label': '2.76'}, {'from': 3, 'to': 17, 'label': '3.54'}, {'from': 3, 'to': 19, 'label': '4.0'}, {'from': 4, 'to': 5, 'label': '1.0'}, {'from'

#### 작성할 파일 제목 정하기
* file_name = "파일명"

In [36]:
file_name = "햄프"

In [49]:
f = open("./results/result_webgraph/data/" + file_name + ".js", 'w', encoding = 'utf8')
f.write(graph_data)
f.close()
print(file_name + ".js 파일 생성 완료")

햄프.js 파일 생성 완료


### 2. keyword.html

In [38]:
html_top = """
<!doctype html>
<html>
<head>
	<title>Knowledge Structure</title>
    """

In [39]:
vis_location = """<script type="text/javascript" src="../../lib/drawing/vis.js"></script>"""

In [40]:
js_location = """<script src="../../results/result_webgraph/data/""" + file_name + """.js" charset="utf-8">
    </script>
    <style type="text/css">#mynetwork 
    {width: 100%;height: 1000px;border: 1px solid lightgray;}</style>
</head>
<body>
"""

In [41]:
title = """<p> Cheonsol of Knowledge Structure.</p>"""

In [42]:
body = """
    <div id="mynetwork"></div>
	<script type="text/javascript">
		var container = document.getElementById('mynetwork');
		var data = { nodes: nodes, edges: edges }; """

In [43]:
option = """
        var options = 
        { "edges": {"smooth": {"forceDirection": "none"}},
        "physics": {"barnesHut": {"centralGravity": 0.5,"springLength": 305},
        "minVelocity": 0.75}}; """

In [44]:
bottom = """var network = new vis.Network(container, data, options);
	</script>
</body>
</html>
"""

In [45]:
html_text = html_top + vis_location + js_location + title + body + option + bottom

In [46]:
html_text

'\n<!doctype html>\n<html>\n<head>\n\t<title>Knowledge Structure</title>\n    <script type="text/javascript" src="../../lib/drawing/vis.js"></script><script src="../../results/result_webgraph/data/햄프.js" charset="utf-8">\n    </script>\n    <style type="text/css">#mynetwork \n    {width: 100%;height: 1000px;border: 1px solid lightgray;}</style>\n</head>\n<body>\n<p> Cheonsol of Knowledge Structure.</p>\n    <div id="mynetwork"></div>\n\t<script type="text/javascript">\n\t\tvar container = document.getElementById(\'mynetwork\');\n\t\tvar data = { nodes: nodes, edges: edges }; \n        var options = \n        { "edges": {"smooth": {"forceDirection": "none"}},\n        "physics": {"barnesHut": {"centralGravity": 0.5,"springLength": 305},\n        "minVelocity": 0.75}}; var network = new vis.Network(container, data, options);\n\t</script>\n</body>\n</html>\n'

In [48]:
html_file = open('./results/result_webgraph/' + file_name + '.html', 'w')
html_file.write(html_text)
html_file.close()
print(file_name + ".html 파일 생성 완료")

햄프.html 파일 생성 완료
